In [ ]:
# Imports gerais

In [ ]:
# Definição de Classes
class Mensagem:
    def __init__(self, mensagem_original, classificacao):
        self.MensagemOriginal = mensagem_original
        self.Classificacao = classificacao
        self.MensagemTratada = None
        self.MensagemTokenizada = None
        self.QuantidadeCaracteres = 0
        self.QuantidadePalavras = 0
        self.QuantidadeVerbos = 0
        self.QuantidadeSubstantivos = 0
        self.ArvoreSintagmas = None

    def ConstroiArvoreSintagmas(self):
        # Lógica para construir a árvore de sintagmas
        pass


class BaseMensagens:
    def __init__(self):
        self.BaseMensagens = []  # Lista de objetos Mensagem

    def Exec(self):
        # Lógica para execução genérica
        pass


class BaseTreinamento(BaseMensagens):
    def __init__(self):
        super().__init__()

    def Exec(self):
        # Lógica específica de treinamento
        pass

class BaseTeste(BaseMensagens):
    def __init__(self):
        super().__init__()
        self.Acuracidade = 0.0

    def Exec(self):
        # Lógica específica de teste
        pass
